# Notebook Web Scraping:  Approach to scrape data from a statistics website

We try to use the data from www.basketball-reference.com and scrape for the respective statitics of the basketball players.
With the gained data we will analyze if a player is underpaid, overpaid or paid fairly in regard to their performance.


## Imports 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import unicodedata
import json, pickle
from bs4 import BeautifulSoup, Comment
import requests
import lxml
import html5lib

## Setting up the scraping for players data


In [2]:
def strip_accents_and_punctuation(text):
    '''preprocessing the player names'''
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')

def update_row_with_dict(df, data, idx):
# updating cells with new entries by using a dictionary
    for key, value in data.items():
        df.at[idx, key] = value


## Scrape player statistics


In [11]:
year=2023

In [12]:
def scrape_player(playerurl, year):
# Scraping data from basketball-reference.com by using the respective URL for each player.
    
    data = {}
    prev_season = f"{year-1}-{year-2000}"
    next_season = f"{year}-{year-2000+1}"
    
    playerresponse = requests.get(playerurl)
    playerpage = playerresponse.text
    playersoup = BeautifulSoup(playerpage, "lxml")

    ## Get additonal data like Height and Weight:
    
    try:
        script_text = playersoup.find('script', {'type': 'application/ld+json'}).getText()
        biodata = json.loads(script_text)
        weight = biodata.get('weight', {}).get('value', '').replace('lbs', '').strip()
        height = biodata.get('height', {}).get('value', '')
    except (AttributeError, KeyError, json.JSONDecodeError):
        weight = np.nan
        height = np.nan
        
    data['Weight']= weight
    data['Height']= height

    ## Getting most common game statistics:
    
    dfpergame = pd.read_html(str(playersoup.find(id='per_game')))[0]
    dfpergame = dfpergame.drop_duplicates(subset=['Season'])
    dfpergame = dfpergame.set_index('Season')

    featurelist = ['Age', 'Tm', 'G', 'GS', 'MP', 'FG%', '3P', '3P%', '3PA', 'FT', 'FT%', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']

    for feature in featurelist:
        try:
            data[feature] = dfpergame.at[prev_season, feature]
        except KeyError:
            data[feature] = np.nan

    ## Get detailed game statistics:  

    dfadvanced = pd.read_html(str(playersoup.find(id='advanced')))[0]
    dfadvanced = dfadvanced.drop_duplicates(subset=['Season'])
    dfadvanced = dfadvanced.set_index('Season')
    
    featurelist = ['USG%', 'TS%', 'PER', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
    
    for feature in featurelist:
        try: 
            data[feature] = dfadvanced.at[prev_season, feature] 
        except KeyError: 
            data[feature] = np.nan

    # Get last year's and next year's salary
    # If a player has played for more than one team, his salary is concatenated

    placeholder = playersoup.select_one('#all_all_salaries .placeholder')
    comment = next(elem for elem in placeholder.next_siblings if isinstance(elem, Comment))
    table = BeautifulSoup(comment, 'lxml')

    dfsalaries = pd.read_html(str(table.find(id="all_salaries")))[0]

    try:
        data['PrevSal'] = dfsalaries.loc[dfsalaries['Season'] == prev_season, 'Salary'].sum()
    except (KeyError, TypeError):
        data['PrevSal'] = np.nan

    try:
        data['NextSal'] = dfsalaries.loc[dfsalaries['Season'] == next_season, 'Salary'].sum()
    except (KeyError, TypeError):
        data['NextSal'] = np.nan
               
    return data

## Example player: Lebron James

In [13]:
playerurl = 'https://www.basketball-reference.com/players/j/jamesle01.html'

fayear = 2023 
data = scrape_player(playerurl, fayear)
print(data)

{'Weight': '250', 'Height': '6-9', 'Age': 38.0, 'Tm': 'LAL', 'G': 55.0, 'GS': 54.0, 'MP': 35.5, 'FG%': 0.5, '3P': 2.2, '3P%': 0.321, '3PA': 6.9, 'FT': 4.6, 'FT%': 0.768, 'FTA': 5.9, 'ORB': 1.2, 'TRB': 8.3, 'AST': 6.8, 'STL': 0.9, 'BLK': 0.6, 'TOV': 3.2, 'PTS': 28.9, 'USG%': 33.3, 'TS%': 0.583, 'PER': 23.9, 'OWS': 3.2, 'DWS': 2.4, 'WS': 5.6, 'WS/48': 0.138, 'OBPM': 5.5, 'DBPM': 0.6, 'BPM': 6.1, 'VORP': 4.0, 'PrevSal': 0, 'NextSal': 0}


## Scrape stats and salaries for all free agents in a given year

In [14]:
def update_row_with_dict(df, data, idx):
    for key in data.keys():
        df.loc[idx, key] = data.get(key)

def scrape_year(year):
    # For calculating if a player is over or underpaid, we look at so called "free agents", as these are players
    # who do not have a contract with a team for the next season. Meaning they will have to negotiate their salary based on their performance. 
    # Giving us a baseline of how much a player should earn for their performance. 

    free_agent_url = 'https://www.basketball-reference.com/contracts/players.html' + str(year)
    # Checking which players do not have a contract for the next season

    prev_season = f"{year-1}-{year-2000}"
    response = requests.get(free_agent_url)
    page = response.text
    FAsoup = BeautifulSoup(page, "lxml")

    table = FAsoup.find('table')
    rows = [row for row in table.find_all('tr')]  # tr tag is for rows
    rows_data = [[td.get_text() for td in row.find_all('td')] for row in rows]

    df = pd.DataFrame()

    for i in range(1, len(rows)):
        try:
            print("Scraping row", i)

            name = rows_data[i][0]
            name_year = f"{name} {year - 1}".replace(' ', '_')  # past year
            name_year = strip_accents_and_punctuation(name_year)

            pos = rows_data[i][1]        # Pos
            fatype = rows_data[i][3]     # Type (UFA, RFA)
            oldteam = rows_data[i][4]    # OTm
            prevstats = rows_data[i][5]  # Previous Year Stats ('Did not play' if didn't play)
            newteam = rows_data[i][7]    # NTm

            nameid = rows[i].find_all('td')[0].find('a')['href']

            data = {
                'Name': name, 'Pos': pos, 'Type': fatype, 'OTm': oldteam,
                'PrevStats': prevstats, 'NTm': newteam, 'ID': nameid
            }
            update_row_with_dict(df, data, name_year)

            playerurl = 'https://www.basketball-reference.com' + nameid
            playerdict = scrape_player(playerurl, year) 
            update_row_with_dict(df, playerdict, name_year)

        except Exception as e:
            print(f"Error occurred for row {i}: {str(e)}")
            continue

    df['PrevYear'] = year - 1
    df.index.name = 'NameYear'

    return df


In [17]:
def update_row_with_dict(df, data, idx):
    for key in data.keys():
        df.loc[idx, key] = data.get(key)

def scrape_year(year):
    free_agent_url = 'https://www.basketball-reference.com/contracts/players.html' + str(year)
    # Checking which players do not have a contract for the next season

    prev_season = f"{year-1}-{year-2000}"
    response = requests.get(free_agent_url)
    page = response.text
    FAsoup = BeautifulSoup(page, "lxml")

    table = FAsoup.find('table')
    if table is None:
        print(f"Table not found for year {year}")
        return pd.DataFrame()

    rows = [row for row in table.find_all('tr')]  # tr tag is for rows
    rows_data = [[td.get_text() for td in row.find_all('td')] for row in rows]

    df = pd.DataFrame()

    for i in range(1, len(rows)):
        try:
            print("Scraping row", i)

            name = rows_data[i][0]
            name_year = f"{name} {year - 1}".replace(' ', '_')  # past year
            name_year = strip_accents_and_punctuation(name_year)

            pos = rows_data[i][1]        # Pos
            fatype = rows_data[i][3]     # Type (UFA, RFA)
            oldteam = rows_data[i][4]    # OTm
            prevstats = rows_data[i][5]  # Previous Year Stats ('Did not play' if didn't play)
            newteam = rows_data[i][7]    # NTm

            nameid = rows[i].find_all('td')[0].find('a')['href']

            data = {
                'Name': name, 'Pos': pos, 'Type': fatype, 'OTm': oldteam,
                'PrevStats': prevstats, 'NTm': newteam, 'ID': nameid
            }
            update_row_with_dict(df, data, name_year)

            playerurl = 'https://www.basketball-reference.com' + nameid
            playerdict = scrape_player(playerurl, year)  # Assuming 'scrape_player' function is defined elsewhere
            update_row_with_dict(df, playerdict, name_year)

        except Exception as e:
            print(f"Error occurred for row {i}: {str(e)}")
            continue

    df['PrevYear'] = year - 1
    df.index.name = 'NameYear'

    return df

# Example usage
df2016 = scrape_year(2017)
df2017 = scrape_year(2018)
df2018 = scrape_year(2019)
df2019 = scrape_year(2020)
df2020 = scrape_year(2021)

#added the following 2 lines of code
df2021 = scrape_year(2022)
df2022 = scrape_year(2023)

df2022.head(3)


Table not found for year 2017
Table not found for year 2018
Table not found for year 2019
Table not found for year 2020
Table not found for year 2021
Table not found for year 2022
Table not found for year 2023


""


## Scrape last years of free agent stats and salaries

In [15]:
df2020 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2020_merged.csv')
df2021 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_merged.csv')
df2022 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')


In [16]:
df2016 = scrape_year(2017)
df2017 = scrape_year(2018)
df2018 = scrape_year(2019)
df2019 = scrape_year(2020)
df2020 = scrape_year(2021)

#added the following 2 lines of code
df2021 = scrape_year(2022)
df2022 = scrape_year(2023)

df2022.head(3)
# Note that for df2020 in particular, NextSal data cannot be obtained this way and is just listed as "0.0".  
# We will obtain in in a different way in Notebook 2 and populate the field in Notebook 3

AttributeError: 'NoneType' object has no attribute 'find_all'

# (2) Scrape Player Stats and Salaries:  Current Year

## Scrape player stats:  current year

In [18]:
##  Per-Game Stats

url_pergame = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'

def scrape_current_season_stats_pergame(url):
    '''Get current season stats for all players from basketball-reference.com
    '''
    data = {}
  
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='all_per_game_stats')))[0]
    return df

##  Advanced Stats

url_advanced = 'https://www.basketball-reference.com/leagues/NBA_2023_advanced.html'

def scrape_current_season_stats_advanced(url):
    '''Get current season stats for all players from basketball-reference.com
    '''
    data = {}
  
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='advanced_stats')))[0]
    return df

dfpergame = scrape_current_season_stats_pergame(url_pergame)
dfadvanced = scrape_current_season_stats_advanced(url_advanced)

## Merge per-game and advanced stats

cols_to_use = dfadvanced.columns.difference(dfpergame.columns)
dfcurrentstats = pd.merge(dfpergame, dfadvanced[cols_to_use], left_index=True, right_index=True, how='outer')
dfcurrentstats = dfcurrentstats.drop(['Unnamed: 19','Unnamed: 24'], axis=1)
print(dfcurrentstats.columns)
dfcurrentstats.sample(5)

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '3PAr',
       'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM', 'ORB%',
       'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS',
       'WS/48'],
      dtype='object')


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
534,405,Julius Randle,PF,28,NYK,77,77,35.5,8.5,18.6,...,5.0,20.3,0.9,11.5,15.4,.581,29.5,3.9,8.1,.142
428,323,Jalen McDaniels,SF,25,PHI,24,3,17.5,2.5,5.1,...,0.4,12.5,2.0,8.6,10.7,.580,16.1,-0.1,1.0,.109
594,449,Marcus Smart,PG,28,BOS,61,61,32.1,4.1,9.9,...,1.0,12.1,2.3,17.9,5.3,.538,17.8,0.5,3.7,.092
267,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
298,225,Bones Hyland,PG,22,DEN,42,1,19.5,4.1,10.3,...,0.1,14.5,1.7,12.5,6.0,.540,28.5,0.3,0.9,.051


## Scrape player salaries:  current year

In [19]:
def scrape_team(team):
    '''Returns dataframe with 2021-22 salary for all players from basketball-reference.com
    '''
    
    url = 'https://www.basketball-reference.com/contracts/' + team + '.html'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='contracts')))[0]
        
    df.columns = df.columns.to_flat_index()
    df.rename(columns= {df.columns[0]: 'Name', df.columns[2]: 'CurrentSalary'}, inplace=True)
    df['CurrentTeam'] = team

    df = df.drop(df.tail(1).index)
    df = df[['Name','CurrentSalary','CurrentTeam']]
    
    df = df.dropna()
    
    return df

test = scrape_team('BOS')
display(test)

,Name,CurrentSalary,CurrentTeam
0,Kristaps Porziņģis,"$36,016,200",BOS
1,Jayson Tatum,"$32,600,060",BOS
2,Jaylen Brown,"$31,830,357",BOS
3,Malcolm Brogdon,"$22,500,000",BOS
4,Derrick White,"$18,357,143",BOS
5,Robert Williams,"$11,571,429",BOS
6,Al Horford,"$10,000,000",BOS
7,Payton Pritchard,"$4,037,277",BOS
8,Luke Kornet,"$2,413,304",BOS
9,Oshae Brissett,"$2,165,000",BOS


In [20]:
tm_to_team =  {
 'TOR': 'Toronto Raptors',         'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',              'BRK': 'Brooklyn Nets',
 'NOP': 'New Orleans Pelicans',    'MIL': 'Milwaukee Bucks',
 'CLE': 'Cleveland Cavaliers' ,    'LAL': 'Los Angeles Lakers',
 'ORL': 'Orlando Magic',           'HOU': 'Houston Rockets' ,
 'WAS': 'Washington Wizards' ,     'PHO': 'Phoenix Suns',
 'UTA': 'Utah Jazz',               'SAC': 'Sacramento Kings',
 'CHO': 'Charlotte Hornets',       'CHI': 'Chicago Bulls' ,
 'NYK': 'New York Knicks',         'DEN': 'Denver Nuggets' ,
 'PHI': 'Philadephia 76ers' ,      'SAS': 'San Antonio Spurs' ,
 'LAC': 'Los Angeles Clippers',    'OKC': 'Oklahoma City Thunder' ,
 'MIN': 'Minnesota Timberwolves',  'DET': 'Detroit Pistons' ,
 'IND': 'Indiana Pacers',          'GSW': 'Golden State Warriors' ,
 'POR': 'Portland Trailblazers',   'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',          'DAL':'Dallas Mavericks',
 }

teams = list(tm_to_team.keys())  #We just need the team names as used in the urls

df = pd.DataFrame(columns=['Name', 'CurrentSalary', 'CurrentTeam'])

for team in teams:
    df_team = scrape_team(team)
    df = pd.concat([df, df_team])
    
df.sample(5)

,Name,CurrentSalary,CurrentTeam
6,Evan Mobley,"$8,882,640",CLE
8,Luke Kornet,"$2,413,304",BOS
13,Kenyon Martin Jr.,"$1,930,681",LAC
9,Ousmane Dieng,"$4,798,440",OKC
14,Jason Preston,"$1,836,096",LAC


We have 5 players who switched teams, therefore occuring twice in the dataframe. We have to delete them.

In [ ]:
# df[df.duplicated(subset = ['Name'])].sort_values("Name")
ids = df["Name"]
df[ids.isin(ids[ids.duplicated()])].sort_values("Name")

In [22]:
duplicate_1 = ((df.Name == 'Evan Mobley') & ~(df.CurrentTeam == 'CLE'))
duplicate_2 = ((df.Name == 'Luka Kornet') & ~(df.CurrentTeam == 'BOS'))
duplicate_3 = ((df.Name == 'Kenyon Martin Jr.') & ~(df.CurrentTeam == 'LAC'))
duplicate_4 = ((df.Name == 'Ousmane Dieng') & ~(df.CurrentTeam == 'OKC'))
duplicate_5 = ((df.Name == 'Jason Preston') & ~(df.CurrentTeam == 'LAC'))

df = df[~duplicate_1 & ~duplicate_2 & ~duplicate_3 & ~duplicate_4 % ~duplicate_5] 

dfcurrentsalaries = df.copy()
dfcurrentsalaries.shape

(0, 3)

## Create player-to-url dictionary


In [26]:
url = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'

player_to_url = {}

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")


table = soup.find('table')
rows = [row for row in table.find_all('tr')]  # tr tag is for rows

for row in rows[1:]:
    try:
        name = row.findAll('td')[0].getText()
        nameid = row.findAll('td')[0].contents[0]['href']
        if name not in player_to_url:
            player_to_url[name] = nameid
    except:
        pass

{k: player_to_url[k] for k in list(player_to_url)[:5]}




{'Precious Achiuwa': '/players/a/achiupr01.html',
 'Steven Adams': '/players/a/adamsst01.html',
 'Bam Adebayo': '/players/a/adebaba01.html',
 'Ochai Agbaji': '/players/a/agbajoc01.html',
 'Santi Aldama': '/players/a/aldamsa01.html'}

In [27]:
dfplayer_to_url = pd.Series(player_to_url).to_frame().reset_index()
dfplayer_to_url.rename(columns= {'index': 'Name',0: 'ID'}, inplace=True)
dfplayer_to_url.head(5)

,Name,ID
0,Precious Achiuwa,/players/a/achiupr01.html
1,Steven Adams,/players/a/adamsst01.html
2,Bam Adebayo,/players/a/adebaba01.html
3,Ochai Agbaji,/players/a/agbajoc01.html
4,Santi Aldama,/players/a/aldamsa01.html


## Saving Data

In [42]:
df.sample(10)
#check for correct data gathering

,Name,CurrentSalary,CurrentTeam
0,Stephen Curry,"$51,915,615",GSW
2,James Harden,"$35,640,000",PHI
4,Derrick White,"$18,107,143",BOS
3,Anthony Edwards,"$13,534,817",MIN
4,Malik Monk,"$9,945,830",SAC
6,Monte Morris,"$9,800,926",DET
2,Jarred Vanderbilt,"$4,698,000",LAL
4,Talen Horton-Tucker,"$11,020,000",UTA
5,Robert Covington,"$11,692,308",LAC
2,Marvin Bagley III,"$12,500,000",DET


In [28]:
df2020.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2020_raw.csv')

#added the following 3 lines of code
df2021.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_raw.csv')
df2022.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_raw.csv')


dfcurrentstats.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfcurrentstats.csv')
df.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfcurrentsalaries.csv')

dfplayer_to_url.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfplayer_to_url.csv')